<a href="https://colab.research.google.com/github/dgansen/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module3-join-and-reshape-data/LS_DS_113_Join_and_Reshape_Data_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [15]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-04-08 22:30:40--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.26.38
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.26.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  45.5MB/s    in 4.7s    

2020-04-08 22:30:45 (41.4 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [16]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [17]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01/instacart_2017_05_01


In [18]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [0]:
import pandas as pd

In [20]:
orders = pd.read_csv('orders.csv')
orders.head()
# I don't need anything

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [21]:
products = pd.read_csv('products.csv')
products.head()
# Need name, id cols

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [22]:
order_products_train = pd.read_csv('order_products__train.csv')
order_products_train.head()
# Need order/product id, add_to_cart_order

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [23]:
order_products_prior = pd.read_csv('order_products__prior.csv')

#Join order_products together
order_products = pd.concat([order_products_prior[['order_id','product_id','add_to_cart_order']],
                           order_products_train[['order_id','product_id','add_to_cart_order']]])
order_products.head()

,order_id,product_id,add_to_cart_order
0,2,33120,1
1,2,28985,2
2,2,9327,3
3,2,45918,4
4,2,30035,5


In [24]:
products.shape

(49688, 4)

In [25]:
# Merge into one DF that matches names to the product IDs
order_products_names = pd.merge(order_products, products[['product_id','product_name']],how='inner',on='product_id')
order_products_names.head()

,order_id,product_id,add_to_cart_order,product_name
0,2,33120,1,Organic Egg Whites
1,26,33120,5,Organic Egg Whites
2,120,33120,13,Organic Egg Whites
3,327,33120,5,Organic Egg Whites
4,390,33120,28,Organic Egg Whites


In [30]:
#Find out how many times each order was added to cart
counts = order_products_names['product_name'].value_counts()
print(counts)

Banana                                                       491291
Bag of Organic Bananas                                       394930
Organic Strawberries                                         275577
Organic Baby Spinach                                         251705
Organic Hass Avocado                                         220877
                                                              ...  
Pure Peppermint Extract                                           1
Frittata, Farmer's Market                                         1
Hot Chili with Beans                                              1
Lowfat Cherry Lime Supernova Kefir Cultured Milk Smoothie         1
Hot Oatmeal Multigrain Raisin                                     1
Name: product_name, Length: 49685, dtype: int64


In [0]:
#Pretend we don't already know our inquiry is the 10 most popular
#Make list of desired items
inq = ['Banana',
       'Bag of Organic Bananas',
       'Organic Strawberries',
       'Organic Baby Spinach',
       'Organic Hass Avocado',
       'Organic Avocado',
       'Large Lemon',
       'Strawberries',
       'Limes',
       'Organic Whole Milk']

In [38]:
#Produce counts list, limited to our interested items
counts.loc[inq]

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

 ## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [41]:
import pandas as pd
import numpy as np

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [42]:
# table2 -> tidy
#reset index
table2 = table2.reset_index()
tidy = table2.melt(id_vars='index')
tidy

,index,variable,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [47]:
# tidy -> table2
print(table2)

tidy.pivot_table(index='index',values='value',columns='variable')

        index  John Smith  Jane Doe  Mary Johnson
0  treatmenta         NaN      16.0           3.0
1  treatmentb         2.0      11.0           1.0


variable,Jane Doe,John Smith,Mary Johnson
index,,,
treatmenta,16.0,NaN,3.0
treatmentb,11.0,2.0,1.0


In [49]:
import seaborn as sns
flights = sns.load_dataset('flights')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [51]:
# Flights Pivot Table
flights.pivot_table(index='year',columns='month')

passengers                       ...                                    
month    January February March April  ... September October November December
year                                   ...                                    
1949         112      118   132   129  ...       136     119      104      118
1950         115      126   141   135  ...       158     133      114      140
1951         145      150   178   163  ...       184     162      146      166
1952         171      180   193   181  ...       209     191      172      194
1953         196      196   236   235  ...       237     211      180      201
1954         204      188   235   227  ...       259     229      203      229
1955         242      233   267   269  ...       312     274      237      278
1956         284      277   317   313  ...       355     306      271      306
1957         315      301   356   348  ...       404     347      305      336
1958         340      318   362   348  ...       404     359      310      337
1959         360      342   406   396  ...       463     407      362      405
1960         417      391   419   461  ...       508     461      390      432

[12 rows x 12 columns]

## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [56]:
#Make dataset containing product/order id, product name, hour of day
stretch1 = pd.merge(order_products[['product_id','order_id']],orders[['order_id','order_hour_of_day']],how='inner',on='order_id')
stretch1 = pd.merge(stretch1,products[['product_id','product_name']],how='inner',on='product_id')
stretch1

,product_id,order_id,order_hour_of_day,product_name
0,33120,2,9,Organic Egg Whites
1,33120,26,16,Organic Egg Whites
2,33120,120,8,Organic Egg Whites
3,33120,327,9,Organic Egg Whites
4,33120,390,12,Organic Egg Whites
...,...,...,...,...
33819101,27499,2269288,14,Non-Dairy Coconut Seven Layer Bar
33819102,27499,2507248,23,Non-Dairy Coconut Seven Layer Bar
33819103,37703,2271346,13,Ultra Sun Blossom Liquid 90 loads Fabric Enhan...
33819104,43725,3004244,18,Sweetart Jelly Beans


In [92]:
#Use previously defined 'counts' to identify popular products
popular = counts[counts>=2900].reset_index().rename(columns={'index':'name','product_name':'count'})
popular

,name,count
0,Banana,491291
1,Bag of Organic Bananas,394930
2,Organic Strawberries,275577
3,Organic Baby Spinach,251705
4,Organic Hass Avocado,220877
...,...,...
2048,Whole Chia Seeds,2902
2049,Organic Spelt Tortillas,2902
2050,3 lb Clementines,2901
2051,Organic Fiber & Protein Pear Blueberry & Spina...,2900


In [94]:
#Filter down stretch1 to only include these foods
stretch1 = stretch1[stretch1['product_name'].isin(popular['name'])]
stretch1

,product_id,order_id,order_hour_of_day,product_name
0,33120,2,9,Organic Egg Whites
1,33120,26,16,Organic Egg Whites
2,33120,120,8,Organic Egg Whites
3,33120,327,9,Organic Egg Whites
4,33120,390,12,Organic Egg Whites
...,...,...,...,...
31956328,16995,3258179,16,Sliced Soppressata Salame
31956329,16995,3296309,13,Sliced Soppressata Salame
31956330,16995,3350177,11,Sliced Soppressata Salame
31956331,16995,3375552,16,Sliced Soppressata Salame


In [105]:
#Since we care about the singular occurrence, not volume of each late item, we filter down to each item's latest time of order
grouped = stretch1.groupby('product_name')
sum(grouped['order_hour_of_day'].max()!=23)


0

**It appears all of the popular items were at one point ordered between 11pm and midnight. A wiser way to go about this analysis might be to compute the average order_hour_of_day for each popular item.**

It is, however, too late into the night.

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####